# Lab 7-1: Tips

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproductibility
torch.manual_seed(1)

## Training and Test Datasets

In [3]:
x_train = torch.FloatTensor([[1, 2, 1],
                            [1, 3, 2],
                            [1, 3, 4],
                            [1, 5, 5],
                            [1, 7, 5],
                            [1, 2, 5],
                            [1, 6, 6],
                            [1, 7, 7]])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [4]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

## Model

In [5]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
        
    def forward(self, x):
        return self.linear(x)

In [6]:
model = SoftmaxClassifierModel()

In [7]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [8]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.cross_entropy(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost: {:6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [9]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
#     print("prediction:", prediction)
#     print("prediction.max(1):", prediction.max(1))
    predicted_classes = prediction.max(1)[1] # dim=1에 대해 max, 그리고 argmax 가져오기
#     print("prediction.max(1)[1]:", predicted_classes)
    correct_count = (predicted_classes == y_test).sum().item()
#     print("before .item():", (predicted_classes == y_test).sum())
#     print("after .item():", (predicted_classes == y_test).sum().item()) # cost.item()처럼 값 가져오기
    cost = F.cross_entropy(prediction, y_test)
    
    print('Accuracy: {}% Cost {:.6f}'.format(
    correct_count / len(y_test) * 100, cost.item()))

In [10]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048377
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [11]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost 1.425844


## Learning Rate

learning rate가 너무 크면 발산하면서 cost가 점점 늘어난다. (overshooting)

In [12]:
model = SoftmaxClassifierModel()

In [13]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [14]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.625000
Epoch    2/20 Cost: 1279135.125000
Epoch    3/20 Cost: 1198379.250000
Epoch    4/20 Cost: 1098825.625000
Epoch    5/20 Cost: 1968197.625000
Epoch    6/20 Cost: 284763.125000
Epoch    7/20 Cost: 1532260.000000
Epoch    8/20 Cost: 1651504.250000
Epoch    9/20 Cost: 521878.437500
Epoch   10/20 Cost: 1397263.125000
Epoch   11/20 Cost: 750986.250000
Epoch   12/20 Cost: 918691.750000
Epoch   13/20 Cost: 1487888.125000
Epoch   14/20 Cost: 1582260.125000
Epoch   15/20 Cost: 685818.000000
Epoch   16/20 Cost: 1140048.750000
Epoch   17/20 Cost: 940566.750000
Epoch   18/20 Cost: 931638.125000
Epoch   19/20 Cost: 1971322.625000


learning rate가 너무 작으면 cost가 거의 줄어들지 않는다.

In [15]:
model = SoftmaxClassifierModel()

In [16]:
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [17]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


따라서 적절한 숫자로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자!

In [18]:
model = SoftmaxClassifierModel()

In [19]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [20]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.341574
Epoch    1/20 Cost: 1.198802
Epoch    2/20 Cost: 1.150877
Epoch    3/20 Cost: 1.131977
Epoch    4/20 Cost: 1.116242
Epoch    5/20 Cost: 1.102514
Epoch    6/20 Cost: 1.089676
Epoch    7/20 Cost: 1.077479
Epoch    8/20 Cost: 1.065775
Epoch    9/20 Cost: 1.054511
Epoch   10/20 Cost: 1.043655
Epoch   11/20 Cost: 1.033187
Epoch   12/20 Cost: 1.023091
Epoch   13/20 Cost: 1.013356
Epoch   14/20 Cost: 1.003968
Epoch   15/20 Cost: 0.994917
Epoch   16/20 Cost: 0.986189
Epoch   17/20 Cost: 0.977775
Epoch   18/20 Cost: 0.969661
Epoch   19/20 Cost: 0.961836


## Data Preprocessing

데이터를 zero-center하고 normalize하자.

In [21]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [22]:
mu = x_train.mean(dim=0)
print(mu)

tensor([84.8000, 84.6000, 85.6000])


In [23]:
sigma = x_train.std(dim=0)
print(sigma)

tensor([11.0544, 12.2393, 12.6214])


In [24]:
norm_x_train = (x_train - mu) / sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


normalize와 zero center한 X로 학습해서 성능을 보자.

In [25]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)

In [26]:
model = MultivariateLinearRegressionModel()

In [27]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [28]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost: {:6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [29]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 57621.960938
Epoch    1/20 Cost: 1115690762240.000000
Epoch    2/20 Cost: 21603375986356781056.000000
Epoch    3/20 Cost: 418311133762336799454134272.000000
Epoch    4/20 Cost: 8099857151294734205918406036684800.000000
Epoch    5/20 Cost:    inf
Epoch    6/20 Cost:    inf
Epoch    7/20 Cost:    inf
Epoch    8/20 Cost:    inf
Epoch    9/20 Cost:    inf
Epoch   10/20 Cost:    inf
Epoch   11/20 Cost:    inf
Epoch   12/20 Cost:    nan
Epoch   13/20 Cost:    nan
Epoch   14/20 Cost:    nan
Epoch   15/20 Cost:    nan
Epoch   16/20 Cost:    nan
Epoch   17/20 Cost:    nan
Epoch   18/20 Cost:    nan
Epoch   19/20 Cost:    nan


## Overfitting

너무 학습 데이터에 한해 잘 학습해 테스트 데이터에 좋은 성능을 내지 못할 수도 있다.

이것을 방지하는 방법은 크게 세 가지인데
1. 더 많은 학습 데이터
2. 더 적은 양의 feature
3. **Regularization**

In [30]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # l2 norm 계산
        l2_reg = 0
        for param in model.parameters():
#             print('param:', param)
#             print('torch.norm(param):', torch.norm(param))
            l2_reg += torch.norm(param)
            
        cost += l2_reg
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost: {:6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [31]:
model = MultivariateLinearRegressionModel()

In [32]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [35]:
train_with_regularization(model, optimizer, norm_x_train, y_train)

param: Parameter containing:
tensor([[ 0.3133, -0.1403,  0.5751]], requires_grad=True)
torch.norm(param): tensor(0.6697, grad_fn=<CopyBackwards>)
param: Parameter containing:
tensor([0.4628], requires_grad=True)
torch.norm(param): tensor(0.4628, grad_fn=<CopyBackwards>)
Epoch    0/20 Cost: 29477.810547
param: Parameter containing:
tensor([[3.7605, 3.2587, 4.0359]], requires_grad=True)
torch.norm(param): tensor(6.4069, grad_fn=<CopyBackwards>)
param: Parameter containing:
tensor([34.4702], requires_grad=True)
torch.norm(param): tensor(34.4702, grad_fn=<CopyBackwards>)
Epoch    1/20 Cost: 18798.513672
param: Parameter containing:
tensor([[5.6179, 5.0300, 5.9076]], requires_grad=True)
torch.norm(param): tensor(9.5792, grad_fn=<CopyBackwards>)
param: Parameter containing:
tensor([61.6762], requires_grad=True)
torch.norm(param): tensor(61.6762, grad_fn=<CopyBackwards>)
Epoch    2/20 Cost: 12059.365234
param: Parameter containing:
tensor([[6.6327, 5.9697, 6.9201]], requires_grad=True)
torch.

torch.norm은 l2 norm을 구해주는 함수로, 유클리디안 거리를 구하는 식과 같다.  
자세한 것은 [링크1](https://light-tree.tistory.com/125)을 참고하면 된다.

L2 규제는 기존의 cost에 규제항을 더해준 것으로, 식은 [링크2](https://dailyheumsi.tistory.com/57)를 참고하면 된다.  
규제항은 파라미터 각각 L2 norm을 구해서 모두 더한 것이다.

In [78]:
a = torch.FloatTensor([[ 0.3133, -0.1403,  0.5751]])

In [79]:
a ** 2

tensor([[0.0982, 0.0197, 0.3307]])

In [86]:
torch.sum(a ** 2)

tensor(0.4486)

In [87]:
torch.sum(a ** 2).sqrt()

tensor(0.6698)

In [88]:
torch.norm(a)

tensor(0.6698)